# Podcast download from GitHub Actions

In [2]:
import requests
import os
import zipfile
import io
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [5]:
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

In [6]:

# Set up the headers for authentication
headers = {
    'Authorization': f'token {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json',
}

# The repository information
owner = 'yanndebray'
repo = 'programming-GPTs'

# List the workflows to get workflow runs
workflows_url = f'https://api.github.com/repos/{owner}/{repo}/actions/runs'
response = requests.get(workflows_url, headers=headers)
workflow_runs = response.json()

In [9]:
len(workflow_runs['workflow_runs'])

30

In [14]:
run0 = workflow_runs['workflow_runs'][0]
artifacts_url = run0['artifacts_url']
artifacts_response = requests.get(artifacts_url, headers=headers)
artifacts = artifacts_response.json()
artifacts['artifacts'][0]

{'id': 1762546077,
 'node_id': 'MDg6QXJ0aWZhY3QxNzYyNTQ2MDc3',
 'name': 'podcast',
 'size_in_bytes': 3398758,
 'url': 'https://api.github.com/repos/yanndebray/programming-GPTs/actions/artifacts/1762546077',
 'archive_download_url': 'https://api.github.com/repos/yanndebray/programming-GPTs/actions/artifacts/1762546077/zip',
 'expired': False,
 'created_at': '2024-08-01T00:04:04Z',
 'updated_at': '2024-08-01T00:04:04Z',
 'expires_at': '2024-10-30T00:01:19Z',
 'workflow_run': {'id': 10189413838,
  'repository_id': 718354569,
  'head_repository_id': 718354569,
  'head_branch': 'main',
  'head_sha': 'a21d3a4ea9c81dca853fc43dc1dba15b20168ed5'}}

In [17]:
download_url = artifacts['artifacts'][0]['archive_download_url']
download_response = requests.get(download_url, headers=headers)
# Create a folder for the run ID if you want to keep them organized
folder_name = f"run_{run0['id']}"
with zipfile.ZipFile(io.BytesIO(download_response.content)) as z:
    # z.extractall(folder_name)
    z.extractall()

Automate

In [18]:
# Function to get all workflow runs with pagination
def get_all_workflow_runs(url):
    runs = []
    while url:
        response = requests.get(url, headers=headers)
        data = response.json()
        runs.extend(data['workflow_runs'])

        # Check if there is a next page
        url = response.links.get('next', {}).get('url', None)
    
    return runs

# URL to list the workflow runs
workflows_url = f'https://api.github.com/repos/{owner}/{repo}/actions/runs'

# Get all workflow runs
workflow_runs = get_all_workflow_runs(workflows_url)

In [20]:
len(workflow_runs)

134

In [23]:
len(os.listdir('../archives/podcast'))

17

In [25]:
# Iterate over each workflow run to list artifacts
for run in workflow_runs[15:]:
    artifacts_url = run['artifacts_url']
    artifacts_response = requests.get(artifacts_url, headers=headers)
    artifacts = artifacts_response.json()

    # Download each artifact
    for artifact in artifacts['artifacts']:
        download_url = artifact['archive_download_url']
        download_response = requests.get(download_url, headers=headers)
        folder_name = f"../archives/podcast"
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        # Iterate over each workflow run to list artifacts
        for run in workflow_runs:
            # Check if the downloaded file is a valid zip file
            if zipfile.is_zipfile(io.BytesIO(download_response.content)):
                # Save the downloaded artifact as a zip file
                with zipfile.ZipFile(io.BytesIO(download_response.content)) as z:
                    z.extractall(folder_name)
                    print(f"Downloaded and extracted artifact {artifact['name']} from run {run['id']}")
            else:
                print("The downloaded file is not a valid zip file.")

Downloaded and extracted artifact podcast from run 10189413838
Downloaded and extracted artifact podcast from run 10171505587
Downloaded and extracted artifact podcast from run 10153669657
Downloaded and extracted artifact podcast from run 10135746639
Downloaded and extracted artifact podcast from run 10127290257
Downloaded and extracted artifact podcast from run 10119063583
Downloaded and extracted artifact podcast from run 10103264395
Downloaded and extracted artifact podcast from run 10085731672
Downloaded and extracted artifact podcast from run 10068071819
Downloaded and extracted artifact podcast from run 10050183125
Downloaded and extracted artifact podcast from run 10032313463
Downloaded and extracted artifact podcast from run 10023973596
Downloaded and extracted artifact podcast from run 10015780350
Downloaded and extracted artifact podcast from run 10000264491
Downloaded and extracted artifact podcast from run 9983151528
Downloaded and extracted artifact podcast from run 99657

KeyboardInterrupt: 